# Prepare dataset

In [ ]:
# autoreload your package
%load_ext autoreload
%autoreload 2
import lie_elicitation_prompts


## Load text dataset

In [ ]:
# load a dataset of paired prompts, to try and get the model to lie
from lie_elicitation_prompts.prompts.prompt_loading import load_preproc_datasets

N = cfg.max_examples
ds_prompts = load_preproc_datasets(
    cfg.datasets,
    N=N,
    seed=cfg.seed,
    num_shots=cfg.num_shots,
)
ds_prompts

## Load tokenized dataset

- tokenize
- filter out truncated
- check which ones the model knows

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

from lie_elicitation_prompts.config import ExtractConfig
cfg = ExtractConfig()

In [ ]:
# quantization_config = BitsAndBytesConfig(load_in_8bit=True)
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)
model = AutoModelForCausalLM.from_pretrained(
    cfg.model,
    device_map="auto",
    quantization_config=quantization_config,
)
tokenizer = AutoTokenizer.from_pretrained(cfg.model)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = "left"
tokenizer.truncation_side = "left"

In [ ]:
from adapter_overseer.helpers.scores import choice2id, choice2ids, row_choice_ids


ds_tokens = ds_prompts.map(
    lambda x: {
        "formatted_chat": tokenizer.apply_chat_template(
            x["messages"], tokenize=False, add_generation_prompt=True
        )
    }
)

ds_tokens = ds_tokens.map(
    lambda x: tokenizer(
        x["formatted_chat"],
        return_tensors="pt",
        max_length=cfg.max_length,
        padding="max_length",
        truncation=True,
    ),
    batched=True,
)


ds_tokens = ds_tokens.map(lambda r: {'choice_ids': row_choice_ids(r, tokenizer)}, desc='choice_ids')


# filter out truncated ones
ds_tokens = ds_tokens.filter(lambda x: x["attention_mask"].sum() < cfg.max_length)
ds_prompts

### Check model knowledge